<a href="https://colab.research.google.com/github/Chetan-Sagathiya/cotton-disease-prediction-deep-learning/blob/master/cotton_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# importing essential libraries

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
from glob import glob

In [3]:
 image_size = [224, 224]

 train_path = '/content/drive/MyDrive/ml_datasets/cotton_disease/data/train'
 valid_path = '/content/drive/MyDrive/ml_datasets/cotton_disease/data/train'

In [4]:
resnet = ResNet50(input_shape = image_size + [3], weights = 'imagenet', include_top=False)

94773248/94765736 [==============================] - 0s 0us/step


In [5]:
# not training existing weights
for layer in resnet.layers:
  layer.trainable = False

In [6]:
# useful in getting total number of classes
folders = glob("/content/drive/MyDrive/ml_datasets/cotton_disease/data/train/*")
folders

['/content/drive/MyDrive/ml_datasets/cotton_disease/data/train/diseased cotton plant',
 '/content/drive/MyDrive/ml_datasets/cotton_disease/data/train/diseased cotton leaf',
 '/content/drive/MyDrive/ml_datasets/cotton_disease/data/train/fresh cotton plant',
 '/content/drive/MyDrive/ml_datasets/cotton_disease/data/train/fresh cotton leaf']

In [7]:
x = Flatten()(resnet.output)

In [8]:
# adding prediction layer
prediction = Dense(len(folders), activation='softmax')(x)

# creating model 
model = Model(inputs = resnet.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [10]:
model.compile(loss = "categorical_crossentropy", optimizer='adam', metrics=['accuracy']
)

In [11]:
# data augmentation
train_datagen = ImageDataGenerator(rescale= 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/ml_datasets/cotton_disease/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [20]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ml_datasets/cotton_disease/data/test',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 18 images belonging to 4 classes.


In [21]:
folders = glob("/content/drive/MyDrive/ml_datasets/cotton_disease/data/test/*")
folders

['/content/drive/MyDrive/ml_datasets/cotton_disease/data/test/diseased cotton leaf',
 '/content/drive/MyDrive/ml_datasets/cotton_disease/data/test/fresh cotton plant',
 '/content/drive/MyDrive/ml_datasets/cotton_disease/data/test/fresh cotton leaf',
 '/content/drive/MyDrive/ml_datasets/cotton_disease/data/test/diseased cotton plant']

In [22]:
# fitting the model 
fit_model = model.fit_generator(
    training_set, 
    validation_data = test_set,
    epochs = 20,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
61/61 [==============================] - 785s 12s/step - loss: 5.7047 - accuracy: 0.3449 - val_loss: 1.3992 - val_accuracy: 0.5556
Epoch 2/20
61/61 [==============================] - 35s 576ms/step - loss: 1.0306 - accuracy: 0.5768 - val_loss: 0.8867 - val_accuracy: 0.7222
Epoch 3/20
61/61 [==============================] - 35s 574ms/step - loss: 0.8825 - accuracy: 0.6580 - val_loss: 0.8513 - val_accuracy: 0.6667
Epoch 4/20
61/61 [==============================] - 35s 577ms/step - loss: 0.9871 - accuracy: 0.5900 - val_loss: 0.8047 - val_accuracy: 0.6667
Epoch 5/20
61/61 [==============================] - 35s 574ms/step - loss: 0.7933 - accuracy: 0.6765 - val_loss: 1.2149 - val_accuracy: 0.5000
Epoch 6/20
61/61 [==============================] - 35s 577ms/step - loss: 0.9243 - accuracy: 0.6328 - val_loss: 1.4311 - val_accuracy: 0.6111
Epoch 7/20
61/61 [==============================] - 35s 574ms/step - loss: 0.8374 - accuracy: 0.6837 - val_loss: 0.5472 - val_accuracy: 0.7778


In [23]:
model.save("/content/drive/MyDrive/ml_datasets/cotton_disease/model_resnet_50.h5")